In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
#Read the ods File
path = "../Raw Data/nov19.ods"
df = pd.read_excel(path, engine="odf")
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,⾸都⼤飯店,Taipei,129,2564,66.25%,2963,7596928,2911542,11167844
1,CapitalHotel,Taipei,0,0,NaN,0,0,0,0
2,國聯⼤飯店,Taipei,243,5110,70.10%,2374,12131282,1929905,15573253
3,UnitedHotel,Taipei,0,0,NaN,0,0,0,0
4,歐華酒店,Taipei,112,2207,65.68%,2920,6444949,6164469,14050614


In [3]:
#Clean Data 
df = pd.DataFrame(df, columns=['Hotel','Area','Room','Demand','Occupancy','ADR','Room_Revenue','F&B_Revenue','Total_Revenue'])
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,⾸都⼤飯店,Taipei,129,2564,66.25%,2963,7596928,2911542,11167844
1,CapitalHotel,Taipei,0,0,NaN,0,0,0,0
2,國聯⼤飯店,Taipei,243,5110,70.10%,2374,12131282,1929905,15573253
3,UnitedHotel,Taipei,0,0,NaN,0,0,0,0
4,歐華酒店,Taipei,112,2207,65.68%,2920,6444949,6164469,14050614


In [4]:
df = df.dropna()
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,⾸都⼤飯店,Taipei,129,2564,66.25%,2963,7596928,2911542,11167844
2,國聯⼤飯店,Taipei,243,5110,70.10%,2374,12131282,1929905,15573253
4,歐華酒店,Taipei,112,2207,65.68%,2920,6444949,6164469,14050614
6,福容⼤飯店台北一館,Taipei,120,2439,67.75%,3030,7388960,11284085,19929603
8,圓⼭⼤飯店,Taipei,500,12315,82.10%,3846,47368119,66249800,129002865
...,...,...,...,...,...,...,...,...,...
238,台東桂⽥喜來登酒店,Others,278,4046,48.51%,2618,10592745,14165667,25531251
240,寶華⼤飯店,Others,78,180,7.69%,1642,295633,0,297633
242,⻑榮桂冠酒店(基隆),Others,140,3625,86.31%,2517,9125765,5833658,17825046
244,鈺通⼤飯店,Others,120,1225,34.03%,1701,2084327,4819418,17825046


In [5]:
df.loc[df['Hotel']=='華泰王⼦⼤飯店']

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
14,華泰王⼦⼤飯店,Taipei,216,5736,88.52%,2940,16865388,10463291,29677232


In [6]:
print(df.dtypes)

Hotel            object
Area             object
Room              int64
Demand            int64
Occupancy        object
ADR               int64
Room_Revenue      int64
F&B_Revenue       int64
Total_Revenue     int64
dtype: object


df['Demand'] = df['Demand'].replace(',', '', regex=True)
df['ADR'] = df['ADR'].replace(',', '', regex=True)
df['Room_Revenue'] = df['Room_Revenue'].replace(',', '', regex=True)
df['F&B_Revenue'] = df['F&B_Revenue'].replace(',', '', regex=True)
df['Total_Revenue'] = df['Total_Revenue'].replace(',', '', regex=True)

In [7]:
df['Occupancy'] = df['Occupancy'].str.rstrip('%').astype('float') / 100.0

In [8]:
df[df['Occupancy'].isnull()]

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue


In [9]:
df.astype({'Room': 'int64',
          'Demand': 'int64',
          'Occupancy': 'int64',
          'ADR': 'float64',
          'Room_Revenue': 'int64',
          'F&B_Revenue': 'int64',
          'Total_Revenue': 'int64',
          }).dtypes

Hotel             object
Area              object
Room               int64
Demand             int64
Occupancy          int64
ADR              float64
Room_Revenue       int64
F&B_Revenue        int64
Total_Revenue      int64
dtype: object

In [10]:
Day = 30

In [11]:
#Calculate/Add RevPAR and Demand
df['RevPAR'] = df['Occupancy']*df['ADR']
df['Supply'] = df['Room']*Day
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
0,⾸都⼤飯店,Taipei,129,2564,0.6625,2963,7596928,2911542,11167844,1962.9875,3870
2,國聯⼤飯店,Taipei,243,5110,0.7010,2374,12131282,1929905,15573253,1664.1740,7290
4,歐華酒店,Taipei,112,2207,0.6568,2920,6444949,6164469,14050614,1917.8560,3360
6,福容⼤飯店台北一館,Taipei,120,2439,0.6775,3030,7388960,11284085,19929603,2052.8250,3600
8,圓⼭⼤飯店,Taipei,500,12315,0.8210,3846,47368119,66249800,129002865,3157.5660,15000
...,...,...,...,...,...,...,...,...,...,...,...
238,台東桂⽥喜來登酒店,Others,278,4046,0.4851,2618,10592745,14165667,25531251,1269.9918,8340
240,寶華⼤飯店,Others,78,180,0.0769,1642,295633,0,297633,126.2698,2340
242,⻑榮桂冠酒店(基隆),Others,140,3625,0.8631,2517,9125765,5833658,17825046,2172.4227,4200
244,鈺通⼤飯店,Others,120,1225,0.3403,1701,2084327,4819418,17825046,578.8503,3600


In [12]:
#Calculate Monthly Room/Supply/Demand/Room Revenue/F&B Revenue/Total Revenue
date = "2019-11-30"
room = round(df["Room"].sum(),0)
supply = round(df["Supply"].sum(),0)
demand = round(df['Demand'].sum(),0)
room_revenue = df["Room_Revenue"].sum()
fb_revenue = df["F&B_Revenue"].sum()
total_revenue = df["Total_Revenue"].sum()

In [13]:
#Calculate Monthly Occupancy/ADR/RevPAR
occ = round((demand/supply),2)
adr = round((room_revenue/demand), 2)
#revpar = round((occ*adr), 2)
revpar = round((room_revenue/supply), 2)

In [14]:
#Append "Total" Row
df = df.append([{'Hotel': 'Total',
                'Area': 'Total',
                'Room': room,
                'Supply': supply,
                'Occupancy': occ,
                'ADR': adr,
                'Room_Revenue': room_revenue,
                'F&B_Revenue': fb_revenue,
                'Total_Revenue': total_revenue,
                'RevPAR': revpar,
                'Demand': demand}], ignore_index=True)
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
120,寶華⼤飯店,Others,78,180,0.0769,1642.00,295633,0,297633,126.2698,2340
121,⻑榮桂冠酒店(基隆),Others,140,3625,0.8631,2517.00,9125765,5833658,17825046,2172.4227,4200
122,鈺通⼤飯店,Others,120,1225,0.3403,1701.00,2084327,4819418,17825046,578.8503,3600
123,陸島酒店,Others,47,801,0.5681,1441.00,1154579,128560,17825046,818.6321,1410
124,Total,Total,28473,625571,0.7300,3774.54,2361239677,2327790894,5309343261,2764.3000,854190


In [15]:
#Add "Date" Column 
df['Date'] = date
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply,Date
120,寶華⼤飯店,Others,78,180,0.0769,1642.00,295633,0,297633,126.2698,2340,2019-11-30
121,⻑榮桂冠酒店(基隆),Others,140,3625,0.8631,2517.00,9125765,5833658,17825046,2172.4227,4200,2019-11-30
122,鈺通⼤飯店,Others,120,1225,0.3403,1701.00,2084327,4819418,17825046,578.8503,3600,2019-11-30
123,陸島酒店,Others,47,801,0.5681,1441.00,1154579,128560,17825046,818.6321,1410,2019-11-30
124,Total,Total,28473,625571,0.7300,3774.54,2361239677,2327790894,5309343261,2764.3000,854190,2019-11-30


In [16]:
df['Date'] = pd.to_datetime(df['Date']).dt.to_period('D')

In [17]:
#Format 'Summary'
df['Date'] = df['Date'].dt.to_timestamp()
#df = df.style.format({'occ':'{:.0%}',
                    #'demand':'{:.0f}',
                    #'adr':'{:.2f}',
                    #'revpar':'{:.2f}'})

In [18]:
print(df.dtypes)

Hotel                    object
Area                     object
Room                      int64
Demand                    int64
Occupancy               float64
ADR                     float64
Room_Revenue              int64
F&B_Revenue               int64
Total_Revenue             int64
RevPAR                  float64
Supply                    int64
Date             datetime64[ns]
dtype: object


#Create DataFrame 'Summary'
summary = pd.DataFrame({
    'date':date,
    'room':room,
    'supply':supply,
    'demand':demand,
    'room_revenue': room_revenue, 
    'fb_revenue': fb_revenue,
    'total_revenue': total_revenue,
    'occ': occ,
    'adr': adr,
    'revpar':revpar   
}, index=[0])
summary

#Format Summary
summary['date'] = pd.to_datetime(summary['date']).dt.to_period('M')
sep20 = summary.style.format({'occ':'{:.0%}',
                    'demand':'{:.0f}',
                    'adr':'{:.2f}',
                    'revpar':'{:.2f}'})

sep20

df.to_excel(r'static\data\2020\sep20.xlsx', index = False, encoding='utf-8-sig')

In [19]:
df.to_csv(r'..\static\data\2019\nov19.csv', index = False, encoding='utf-8-sig')

In [20]:
from sqlalchemy import create_engine

In [21]:
connection_string = "postgres:postgres@localhost:5432/hotel_db"
engine = create_engine(f'postgresql://{connection_string}', encoding = 'utf-8')

In [22]:
engine.table_names()

['jan20',
 'oct20',
 'nov20',
 'dec20',
 'combine20',
 'dec19',
 'nov19',
 'oct19',
 'sep19',
 'jun19',
 'aug19',
 'jul19',
 'may19',
 'apr19',
 'mar19',
 'feb19',
 'jan19',
 'feb20',
 'mar20',
 'apr20',
 'may20',
 'jun20',
 'jul20',
 'aug20',
 'sep20']

In [23]:
df.to_sql(name='nov19', con=engine, if_exists='append', index=False)